# 决策树C4.5算法

## 一、简介

C4.5基于ID3算法改进得到，ID3算法对可取值数量较多的属性有所偏好。例：变'编号属性'，对应的信息增益就很大。
为了避免这种现象，C4.5算法不采用信息增益而采用 `增益率` 来选择最后划分属性。

## 二、算法思路

### 2.1 考虑连续属性
&nbsp;&nbsp;&nbsp;&nbsp;C4.5的思路是将连续的特征离散化。比如m个样本的连续特征a有m个，从小到大排列为 $ a_{1}, a_{2}, ... ,a_{m} $ 则C4.5取相邻两样本值的平均数，一共取得m-1个划分点，其中第i个划分点 $ T_{i} $ 表示为：$ T_{i} = \frac{a_{1} + a_{2} + 1}{2} $。对于这m-1个点，分别计算以该点作为二元分类点时的信息增益。选择信息增益最大的点作为该连续特征的二元离散分类点。比如取到的增益最大的点为 $a_{t}$ , 则小于 $a_{t}$ 的值为类别1，大于 $a_{t}$ 的值为类别2，这样我们就做到了连续特征的离散化。要注意的是，与离散属性不同的是，如果当前节点为连续属性，则该属性后面还可以参与子节点的产生选择过程。

### 2.2 对取值多属性的划分偏好

&nbsp;&nbsp;&nbsp;&nbsp;不直接使用ID3中的信息增益，而是使用 `增益率` 来选择最优划分属性。增益率定义为：

$$ Gain\_ratio(D, a) = \frac{Gain(D, a)}{IV(a)} $$

其中，

$$ IV(a) = - \sum_{v=1}^{V} \frac{|D^{v}|}{|D|} \log_{2} \frac{|D^{v}|}{|D|} $$

&nbsp;&nbsp;&nbsp;&nbsp;特征数越多的特征对应的特征熵越大，它作为分母，可以校正信息增益容易偏向于取值较多的特征的问题。

### 2.3 缺失值处理

&nbsp;&nbsp;&nbsp;&nbsp;缺失值处理的问题，主要需要解决的是两个问题，一是在样本某些特征缺失的情况下选择划分的属性，二是选定了划分属性，对于在该属性上缺失特征的样本的处理。

1. 对于第一个子问题，对于某一个有缺失特征值的特征A。C4.5的思路是将数据分成两部分，对每个样本设置一个权重（初始可以都为1），然后划分数据，一部分是有特征值A的数据D1，另一部分是没有特征A的数据D2. 然后对于没有缺失特征A的数据集D1来和对应的A特征的各个特征值一起计算加权重后的信息增益比，最后乘上一个系数，这个系数是无特征A缺失的样本加权后所占加权总样本的比例。

2. 对于第二个子问题，可以将缺失特征的样本同时划分入所有的子节点，不过将该样本的权重按各个子节点样本的数量比例来分配。比如缺失特征A的样本a之前权重为1，特征A有3个特征值A1,A2,A3。 3个特征值对应的无缺失A特征的样本个数为2,3,4.则a同时划分入A1，A2，A3。对应权重调节为2/9,3/9, 4/9。

### 2.4 过拟合处理

可通过剪枝方法进行处理，在后期CART笔记中详细描述

In [19]:
import numpy as np
import pandas as pd

from utils.dataset import load_watermelon

from collections import Counter


def iv(d:int, df: pd.DataFrame, a: str) -> float:
    """获取属性的固有属性

    Args:
        d (int): [总样本数量]
        df (pd.DataFrame): [样本]
        a (str): [属性a的列名]

    Returns:
        float: [iv值]
    """
    a_unique_values = df[a].unique()

    iv_value = 0
    for a_value in a_unique_values:
        v_df = df.loc[df[a]==a_value]
        dv = v_df.shape[0]
        _iv = -1 * (dv / d) * np.log2(dv / d)
        iv_value += _iv
    
    # iv
    return iv_value


def gain_ratio(gain: float, iv: float) -> float:
    """获取信息增益率

    Args:
        gain (float): [样本d 属性a 的信息增益]
        iv (float): [属性a的固有属性]

    Returns:
        float: [增益率]
    """
    return gain / iv


def ent(y: np.array) -> float:
    """获取样本集的信息熵

    Args:
        y (np.array): [标签集合]

    Returns:
        float: [信息熵]
    """
    target_counter = Counter(y)
    target_number = y.shape[0]

    ent_value = 0
    for _, key in enumerate(target_counter.keys()):
        _ent = -1 * (target_counter[key] / target_number) * (np.log2(target_counter[key] / target_number))
        ent_value +=_ent
    return ent_value


def gain(d_number: int, d_ent: float, df: pd.DataFrame, a: str) -> float:
    """获取信息增益

    Args:
        d_number (int): [整体样本数量]
        d_ent (float): [整体样本的信息熵]
        df (pd.DataFrame): [结点样本]]
        a (str): [划分属性a]

    Returns:
        float: [节点样本 对应 a属性划分的信息增益]
    """
    a_unique_values = df[a].unique()

    dv_ent_value = 0
    for a_value in a_unique_values:
        v_df = df.loc[df[a]==a_value]
        weight = v_df.shape[0] / d_number
        dv_ent = weight * ent(v_df['target'])
        dv_ent_value += dv_ent
    
    # gain
    return d_ent - dv_ent_value

# ------------------------------------------------------


class C45(object):
    def __init__(self, df) -> None:
        self.df = df
        self.shape = df.shape
        self.tree = list()
        self.d_ent = ent(self.df['target'])
        super().__init__()
    
    def separation(self, df, parent='root'):
        """进行划分

        Args:
            df ([type]): [节点样本]
            parent (str, optional): [该节点对应的父节点名称]. Defaults to 'root'.

        Returns:
            [type]: [若无数据或数据只有一个target值时，返回1]
        """
        X_df = df.loc[:, ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']]
        y = df['target']

        if X_df.shape[0] == 0:
            # 若无数据输入
            return 1
        elif y.nunique() == 1:
            # 若多类型
            return 1

        max_gain_radio = 0
        res_dict = {}
        for _, col in enumerate(X_df.columns):  # 对每个属性进行划分
            gain_a = gain(
                self.shape[0], self.d_ent, df, 
                col
            )
            iv_a = iv(self.shape[0], df, col)
            gain_radio_a = gain_ratio(gain_a, iv_a)
            if gain_radio_a > max_gain_radio:
                max_gain_radio = gain_radio_a
                if col != parent.split('-')[0]:
                    res_dict['gain_radio'] = gain_radio_a
                    res_dict['a'] = col
                    res_dict['parent'] = parent
        
        # 若无res，则返回1
        if not 'a' in list(res_dict.keys()):
            return 1
        
        # 继续划分
        a_unique_values = df[res_dict['a']].unique()
        for a_value in a_unique_values:
            v_df = df.loc[df[res_dict['a']]==a_value]
            res_dict['data'] = list(v_df.index)
            res_dict['a_v'] = res_dict['a'] + '-' + a_value

            # update self.tree
            print(res_dict['a_v'], parent)
            rv = self.separation(v_df, parent=res_dict['a'] + '-' + a_value)
            if rv == 1:
                self.tree.append({'a': res_dict['a'], 'parent': res_dict['parent']})
    
    def run(self, ):
        """ID3划分选择

        Returns:
            [list]: [划分的顺序关系]
        """
        self.separation(self.df)
        return self.tree



In [20]:
watermelon_df = load_watermelon()
X = watermelon_df.loc[:, ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']]
y = watermelon_df['target']

separation_res = C45(watermelon_df).run()
print("划分规则 连接关系 -----> ")
print(separation_res)

纹理-清晰 root
根蒂-蜷缩 纹理-清晰
根蒂-稍蜷 纹理-清晰
色泽-青绿 根蒂-稍蜷
色泽-乌黑 根蒂-稍蜷
根蒂-硬挺 纹理-清晰
纹理-稍糊 root
根蒂-稍蜷 纹理-稍糊
色泽-乌黑 根蒂-稍蜷
色泽-青绿 根蒂-稍蜷
色泽-浅白 根蒂-稍蜷
根蒂-蜷缩 纹理-稍糊
纹理-模糊 root
划分规则 连接关系 -----> 
[{'a': '根蒂', 'parent': '纹理-清晰'}, {'a': '色泽', 'parent': '根蒂-稍蜷'}, {'a': '色泽', 'parent': '根蒂-稍蜷'}, {'a': '根蒂', 'parent': '纹理-清晰'}, {'a': '色泽', 'parent': '根蒂-稍蜷'}, {'a': '色泽', 'parent': '根蒂-稍蜷'}, {'a': '色泽', 'parent': '根蒂-稍蜷'}, {'a': '根蒂', 'parent': '纹理-稍糊'}, {'a': '纹理', 'parent': 'root'}]
